In [171]:
path='C:\\Users\\umairansari\\Desktop\\comp data\\analytics vidya\\janata hack'
import os
os.chdir(path)
print(os.getcwd())
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
os.listdir()

C:\Users\umairansari\Desktop\comp data\analytics vidya\janata hack


['test_KaymcHn.csv', 'train_jqd04QH.csv']

In [172]:
trained=pd.read_csv("train_jqd04QH.csv")
test=pd.read_csv("test_KaymcHn.csv")
trained["flag"]="train"
test["flag"]="test"

In [173]:
train=pd.concat([trained,test])

C:\Users\umairansari\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [174]:
# import missingno as msno
# msno.bar(train)

In [175]:
for i in train.columns:
    x=train[i].isna().sum()
    print("missing values in {} are {}%".format(i,(x/train.shape[0])*100))

missing values in city are 0.0%
missing values in city_development_index are 0.0%
missing values in company_size are 26.452965847813058%
missing values in company_type are 28.067705212702215%
missing values in education_level are 2.5524266027561415%
missing values in enrolled_university are 1.860395446375075%
missing values in enrollee_id are 0.0%
missing values in experience are 0.3085680047932894%
missing values in flag are 0.0%
missing values in gender are 22.426602756141403%
missing values in last_new_job are 2.0101857399640504%
missing values in major_discipline are 15.67106051527861%
missing values in relevent_experience are 0.0%
missing values in target are 45.0%
missing values in training_hours are 0.0%


In [176]:
#unique_values
for i in train.columns:
    print("unique values in {} are {} ".format(i,len(train[i].unique())))

unique values in city are 123 
unique values in city_development_index are 93 
unique values in company_size are 9 
unique values in company_type are 7 
unique values in education_level are 6 
unique values in enrolled_university are 4 
unique values in enrollee_id are 33380 
unique values in experience are 23 
unique values in flag are 2 
unique values in gender are 4 
unique values in last_new_job are 7 
unique values in major_discipline are 7 
unique values in relevent_experience are 2 
unique values in target are 3 
unique values in training_hours are 241 


# DATA CLEANING

In [177]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33380 entries, 0 to 15020
Data columns (total 15 columns):
city                      33380 non-null object
city_development_index    33380 non-null float64
company_size              24550 non-null object
company_type              24011 non-null object
education_level           32528 non-null object
enrolled_university       32759 non-null object
enrollee_id               33380 non-null int64
experience                33277 non-null object
flag                      33380 non-null object
gender                    25894 non-null object
last_new_job              32709 non-null object
major_discipline          28149 non-null object
relevent_experience       33380 non-null object
target                    18359 non-null float64
training_hours            33380 non-null int64
dtypes: float64(2), int64(2), object(11)
memory usage: 4.1+ MB


In [178]:
train.isna().sum()

city                          0
city_development_index        0
company_size               8830
company_type               9369
education_level             852
enrolled_university         621
enrollee_id                   0
experience                  103
flag                          0
gender                     7486
last_new_job                671
major_discipline           5231
relevent_experience           0
target                    15021
training_hours                0
dtype: int64

In [179]:
#---------------------------------------------------------------------------------
train["city"]=train["city"].str.replace("city_","")
train["city"]=pd.to_numeric(train["city"],errors='coerce')

#---------------------------------------------------------------------------------

train["company_type"]=train["company_type"].fillna("Other1")
train["company_type"]=train["company_type"].map({'Pvt Ltd':0, 'Funded Startup':1, 'Public Sector':2, 'Other1':6,
                                                 'Early Stage Startup':3, 'NGO':4, 'Other':5})

#---------------------------------------------------------------------------------
train["gender"]=train["gender"].fillna("other")
train["gender"]=train["gender"].map({'Male':1,'other':3,'Female':0,'Other':2})


#---------------------------------------------------------------------------------

train["company_size"]=train["company_size"].str.replace("/","0")
train["company_size"]=train["company_size"].str.replace("<","5-")
train["company_size"]=train["company_size"].fillna("unknown")

#---------------------------------------------------------------------------------
train["relevent_experience"]=train["relevent_experience"].map({'Has relevent experience':1,'No relevent experience':0})


#---------------------------------------------------------------------------------
# cleaning education level
train["education_level"]=train["education_level"].fillna("Other_Education_or_not_educated")
train["education_level"]=train["education_level"].map({'Graduate':3, 'Masters':4, 'High School':2, 'Phd':5,
       'Other_Education_or_not_educated':0, 'Primary School':1})

#---------------------------------------------------------------------------------
#filling na  of major with no major because people who have not been graduated are not eligible for majors
#people who study less than garduate are not eligible for majors
trained_m=train[train["education_level"]<3]
print(trained_m.shape)
print(train["major_discipline"].isna().sum())
train["major_discipline"]=train["major_discipline"].fillna("No Major")

#---------------------------------------------------------------------------------

#cleaning experience level and filling na
train["experience"]=train["experience"].str.replace(">20","25")
train["experience"]=train["experience"].str.replace("<1","0")
train["experience"]=pd.to_numeric(train["experience"],errors='coerce')
train["experience"]=train["experience"].fillna(0)

#---------------------------------------------------------------------------------
train["enrolled_university"]=train["enrolled_university"].fillna("no_enrollment")
train["enrolled_university"]=train["enrolled_university"].map({'no_enrollment':0,
                                                               'Full time course':1,
                                                               'Part time course':2})

#---------------------------------------------------------------------------------
train["last_new_job"]=train["last_new_job"].str.replace("never","0")

train["last_new_job"]=train["last_new_job"].str.replace(">4","5")
train["last_new_job"]=train["last_new_job"].fillna("0")
train["last_new_job"]=pd.to_numeric(train["last_new_job"],errors='coerce')

#---------------------------------------------------------------------------------
newdata=pd.get_dummies(data=train,columns=['company_size','major_discipline'],prefix="dum")
#---------------------------------------------------------------------------------

#saperating our data
print(trained.shape)
trained=newdata[newdata["flag"]=='train']
test=newdata[newdata["flag"]=='test']
print(trained.shape,test.shape)


(5190, 15)
5231


# SOME EDA